## Libraries

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
path_data='./data/'
name_file_lieux_equipements_culturels = 'nbre_equipement_parcommuneetcodepostal_ET_LatitudeLongitude_ET_POPULATION.csv'

## Load data

In [4]:
# latitude, longitude, Population
df_synthese=pd.read_csv(path_data+name_file_lieux_equipements_culturels, sep='\t', on_bad_lines='skip' ,encoding='utf-8',low_memory=False)
df_synthese.head()

,PTOT,latitude,longitude,code_insee,Type équipement ou lieu,count,categorie
0,848.0,NaN,NaN,01001,Monument,1,patrimoine
1,15240.0,NaN,NaN,01004,Bibliothèque,1,vivant
2,15240.0,NaN,NaN,01004,Cinéma,1,vivant
3,15240.0,NaN,NaN,01004,Monument,3,patrimoine
4,15240.0,NaN,NaN,01004,Service d'archives,1,patrimoine


In [9]:
code_commune = df_synthese['code_insee'].unique()
code_commune

array(['01001', '01004', '01005', ..., '98831', '98832', '99999'],
      dtype=object)

In [10]:
id = 1
series = df_synthese.loc[df_synthese['code_insee']==code_commune[id],'Type équipement ou lieu']
series

1          Bibliothèque
2                Cinéma
3              Monument
4    Service d'archives
Name: Type équipement ou lieu, dtype: object

In [6]:
series.values

array(['Bibliothèque', 'Cinéma', 'Monument', "Service d'archives"],
      dtype=object)

In [7]:
series.index

Index([1, 2, 3, 4], dtype='int64')

In [11]:
px.bar(series,x=series.values,y=series.index)